In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)
# from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

2.0.0-rc1


In [2]:
(data, label),_ = keras.datasets.mnist.load_data()

In [4]:
print(data.shape)
print(label.shape)
print(data.min())
print(data.max())

(60000, 28, 28)
(60000,)
0
255


In [5]:
data = tf.convert_to_tensor(data, dtype=tf.float32) / 255.

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((data, label))

In [7]:
dataset = dataset.batch(32).repeat(10)

In [9]:
model = keras.Sequential([keras.layers.Dense(256, activation='relu'),
                          keras.layers.Dense(256, activation='relu'),
                          keras.layers.Dense(256, activation='relu'),
                          keras.layers.Dense(10)])
model.build(input_shape=(None, 28*28))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  65792     
_________________________________________________________________
dense_2 (Dense)              multiple                  65792     
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = keras.optimizers.SGD(lr=0.01)
acc = keras.metrics.Accuracy()

In [16]:
for step, (x,y) in enumerate(dataset):
    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        y_hat = model(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # [b, 10]
        loss = tf.square(y_hat - y_onehot)
        # [b]
        loss = tf.reduce_sum(loss) / 32

    acc.update_state(tf.argmax(y_hat, axis=1), y)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if step % 200==0:
        print("step={:0>6d}\tloss={:.6f}\tacc={:.4f}".format(step,loss,acc.result().numpy()))
        acc.reset_states()

step=000000	loss=0.085429	acc=0.9786
step=000200	loss=0.071440	acc=0.9786
step=000400	loss=0.078449	acc=0.9759
step=000600	loss=0.068370	acc=0.9752
step=000800	loss=0.068183	acc=0.9766
step=001000	loss=0.105982	acc=0.9731
step=001200	loss=0.077731	acc=0.9777
step=001400	loss=0.066634	acc=0.9762
step=001600	loss=0.061008	acc=0.9711
step=001800	loss=0.082098	acc=0.9775
step=002000	loss=0.077317	acc=0.9812
step=002200	loss=0.023894	acc=0.9780
step=002400	loss=0.069606	acc=0.9756
step=002600	loss=0.081268	acc=0.9761
step=002800	loss=0.044286	acc=0.9769
step=003000	loss=0.056662	acc=0.9784
step=003200	loss=0.091101	acc=0.9764
step=003400	loss=0.053910	acc=0.9750
step=003600	loss=0.037452	acc=0.9745
step=003800	loss=0.089894	acc=0.9809
step=004000	loss=0.068468	acc=0.9819
step=004200	loss=0.069973	acc=0.9767
step=004400	loss=0.067486	acc=0.9777
step=004600	loss=0.112071	acc=0.9778
step=004800	loss=0.065653	acc=0.9780
step=005000	loss=0.055686	acc=0.9786
step=005200	loss=0.135092	acc=0.9761
s